## 🛳️ Validación de Datos en Titanic con DQX (Regla SQL)  

**Dataset usado:** [Titanic_Train.csv](https://raw.githubusercontent.com/lucasmengual92/datasets/main/data/titanic_train.csv)  

**Regla SQL:** La columna `Fare` debe ser mayor que cero.

**Regla BONUS:** "La columna `Embarked` deben tener valores `C`, `Q`, o `S`.

**Basado en:** [DQX-Data Quality Framework](https://databrickslabs.github.io/dqx/)

**Pre-requisitos:** Tener una cuenta/workspace en Databricks.

### 0. Instalacion PYPI y Restart Kernel
- Si el la 1ra vez:
  - !pip install databricks-labs-dqx
- Muy probable que tendras que reiniciar el kernel de tu entorno Python para que DQX se instale correctacmete.

In [ ]:
!pip install databricks-labs-dqx

In [ ]:
dbutils.library.restartPython()

### 1. Cargar el Dataset de Titanic

In [ ]:
import yaml
import pandas as pd
from databricks.labs.dqx.engine import DQEngine
from databricks.sdk import WorkspaceClient

In [ ]:
url = "https://raw.githubusercontent.com/lucasmengual92/datasets/main/data/titanic_train.csv"

df = pd.read_csv(url, delimiter=",", header=0)
sdf = spark.createDataFrame(df)

In [ ]:
# Mostrar esquema y ejemplo de datos
print("Esquema del dataset:")
sdf.printSchema()
print("\nMuestra de datos:")
display(sdf)

### 2. Definir Regla de Calidad con DQX

In [ ]:
# Definir las Reglas de Calidad (YAML)
dqx_checks = yaml.safe_load("""
- check:
    function: sql_expression
    arguments:
      expression: Fare <= 0
      msg: Fare es menor o igual que 0
  criticality: error
- check:
    function: value_is_in_list
    arguments:
      col_names:
        - Embarked
      allowed:
      - C
      - Q
      - S
  criticality: error
  """)

# validate the checks
status = DQEngine.validate_checks(dqx_checks)
assert not status.has_errors

dq_engine = DQEngine(WorkspaceClient())

# apply quality checks
safe_df, quarantine_df = dq_engine.apply_checks_by_metadata_and_split(sdf, dqx_checks)

### 3. Resultados

In [ ]:
print(f"► Filas válidas: {safe_df.count()}")
print(f"► Filas inválidas: {quarantine_df.count()}")

In [ ]:
display(quarantine_df)